In [10]:
print("Setting CUDA devices...")
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="5"


from fastai.conv_learner import *
from fastai.dataset import *

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split


Setting CUDA devices...


In [11]:
DATASET_ROOT = '/media/data-nvme/dev/datasets/airbus/'
PATH = DATASET_ROOT
TRAIN = DATASET_ROOT + 'train_v2/'
TEST = DATASET_ROOT + 'test_v2/'
SEGMENTATION = DATASET_ROOT + 'train_ship_segmentations_v2.csv'
exclude_list = ['6384c3e78.jpg'] #corrupted image
PRETRAINED = DATASET_ROOT + 'models/resnet34s256_kaggle-airbus-l0.053-a0.981.h5'

In [28]:
#!ls {TRAIN}

In [12]:
!rm -r {PATH}tmp # do that to avoid

In [13]:
nw = 12   #number of workers for data loader
arch = resnet34 #specify target architecture

In [14]:
train_names = [f for f in os.listdir(TRAIN)]
test_names = [f for f in os.listdir(TEST)]
for el in exclude_list:
    if(el in train_names): train_names.remove(el)
    if(el in test_names): test_names.remove(el)
#5% of data in the validation set is sufficient for model evaluation
tr_n, val_n = train_test_split(train_names, test_size=0.05, random_state=42)

In [15]:
class pdFilesDataset(FilesDataset):
    def __init__(self, fnames, path, transform):
        self.segmentation_df = pd.read_csv(SEGMENTATION).set_index('ImageId')
        super().__init__(fnames, transform, path)
    
    def get_x(self, i):
        img = open_image(os.path.join(self.path, self.fnames[i]))
        if self.sz == 768: return img 
        else: return cv2.resize(img, (self.sz, self.sz))
    
    def get_y(self, i):
        if(self.path == TEST): return 0
        masks = self.segmentation_df.loc[self.fnames[i]]['EncodedPixels']
        if(type(masks) == float): return 0 #NAN - no ship 
        else: return 1
    
    def get_c(self): return 2 #number of classes

In [16]:
def get_data(sz,bs):
    #data augmentation
    aug_tfms = [RandomRotate(20, tfm_y=TfmType.NO),
                RandomDihedral(tfm_y=TfmType.NO),
                RandomLighting(0.05, 0.05, tfm_y=TfmType.NO)]
    tfms = tfms_from_model(arch, sz, crop_type=CropType.NO, tfm_y=TfmType.NO, 
                aug_tfms=aug_tfms)
    ds = ImageData.get_ds(pdFilesDataset, (tr_n[:-(len(tr_n)%bs)],TRAIN), 
                (val_n,TRAIN), tfms, test=(test_names,TEST))
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
#     md.is_multi = False
    return md

In [17]:
sz = 256 #image size
bs = 64  #batch size

md = get_data(sz,bs)

In [19]:
# learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
# learn.opt_fn = optim.Adam

In [ ]:
#learn.lr_find()
#learn.sched.plot()

In [ ]:
#learn.fit(2e-3, 1)

In [20]:
# learn.unfreeze()
# lr=np.array([1e-4,5e-4,2e-3])

In [21]:
def save_when_acc(self, metrics):
    loss, acc = metrics[0], metrics[1]
    filename = "{}-l{:.3f}-a{:.3f}".format(self.name, loss[0], acc)
    if self.best_acc == None or acc > self.best_acc:
        self.best_acc = acc
        self.best_loss = loss
        self.model.save(f'{filename}')
    elif acc == self.best_acc and  loss < self.best_loss:
        self.best_loss = loss
        self.model.save(f'{filename}')
SaveBestModel.save_when_acc = save_when_acc

In [22]:
#learn.fit(lr, 1, cycle_len=2, use_clr=(20,8), best_save_name=arch.__name__ + "_kaggle-airbus")

In [23]:
#learn.sched.plot_lr()

In [24]:
#learn.save('Resnet34_lable_256_1')

## Do many times

In [25]:
# A tester :
'''
[1, 2, (20,8), 384, 32],
'''
training_loop = [
    [1, 1, (40,10), 64, 256],
    [1, 1, (40,10), 128, 128],
    [1, 1, (40,10), 256, 64],
    [1, 1, (40,10), 256, 64],
    [1, 1, (40,10), 384, 32],
    [1, 1, (40,10), 768, 16]
]
i = 0
md = get_data(sz,bs)
learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
learn.opt_fn = optim.Adam
learn.load(PRETRAINED)
#learn.fit(2e-3, 1, best_save_name='ResNet34_s' + str(sz) + "_kaggle-airbus")
learn.unfreeze()
lr=np.array([1e-4,5e-4,2e-3])
for epochs, cycle_len, use_clr, sz, bs in training_loop:
    i+=1
    md = get_data(sz,bs)
    learn.set_data(md)
    learn.fit(lr, epochs, cycle_len=cycle_len, use_clr=use_clr,
        best_save_name='ResNet34_s' + str(sz) + "_kaggle-airbus" + str(i))
    print("----------", i, "------------------------------")

KeyError: 'Idx 6 is out-of-bounds'

## Load the best

In [ ]:
log_preds,y = learn.predict_with_targs(is_test=True)
probs = np.exp(log_preds)[:,1]
pred = (probs > 0.5).astype(int)

In [ ]:
df = pd.DataFrame({'id':test_names, 'p_ship':probs})
df.to_csv('ship_detection.csv', header=True, index=False)

In [ ]:
sz = 384 #image size
bs = 32  #batch size

md = get_data(sz,bs)
learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
learn.opt_fn = optim.Adam
learn.unfreeze()
lr=np.array([1e-4,5e-4,2e-3])